In [1]:
!pip install TTS
!pip install pyarrow

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached TTS-0.22.0-cp310-cp310-manylinux1_x86_64.whl.metadata (21 kB)
  Using cached soundfile-0.13.1-py2.py3-none-manylinux_2_28_x86_64.whl.metadata (16 kB)
  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached inflect-7.5.0-py3-none-any.whl.metadata (24 kB)
  Using cached anyascii-0.3.3-py3-none-any.whl.metadata (1.6 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pysbd-0.3.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached umap_learn-0.5.9.post2-py3-none-any.whl.metadata (25 kB)
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached trainer-0.0.36-py3-none-any.whl.metadata (8.1 kB)
  Using cached coqpit-0.0.17-py3-none-any.whl.metadata (11 kB)
  Using cached jieba-0.42.1-py3-none-any.whl
  Using cached pypinyin-0.55.0-py2.py3-none-any.whl.metadata (12 

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import random
from TTS.api import TTS # needs Python 3.11 or older
from ask_lisa import ask_lisa

In [10]:
base_dir = "asr_bundestag_clean"
train = base_dir + "/train_nodev"
validate = base_dir + "/train_dev"
test = base_dir + "/test"

In [11]:
mapping = {}

with open(train + "/wav.scp", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split()
        mapping.update({words[0]: [words[1]]})

for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    print(key, ": ", value)

7535988_0_172_12 :  ['wavs/7535988_0_172_12.wav']
7536262_0_15_7 :  ['wavs/7536262_0_15_7.wav']
7546736_0_163_2 :  ['wavs/7546736_0_163_2.wav']
7511688_0_142_16 :  ['wavs/7511688_0_142_16.wav']
7535129_3_253 :  ['wavs/7535129_3_253.wav']


In [12]:
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split(maxsplit=1)
        mapping[words[0]].append(words[1])

for i, (key, value) in enumerate(mapping.items()):
    if i >= 2:
        break
    print(key, ": ", value)

7535988_0_172_12 :  ['wavs/7535988_0_172_12.wav', 'das bedeutet jetzt dass die länder auf jeden fall daten erheben müssen']
7536262_0_15_7 :  ['wavs/7536262_0_15_7.wav', 'kein geschenk das ist verdient meine damen und herren']


In [13]:
word_count = {}
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        count = len(line.strip().split()) - 1
        if count in word_count:
            word_count[count] = word_count[count] + 1
        else:
            word_count[count] = 1

counts = sorted(word_count.keys())
frequencies = [word_count[c] for c in counts]

plt.bar(counts, frequencies)
plt.xlabel("Wörter pro Transkript")
plt.ylabel("Häufigkeit")
plt.show()

In [14]:
filtered_mapping = {}
for key, value in mapping.items():
    count = len(value[1].split())
    if 70 < count < 1000:
        filtered_mapping.update({key: value})

mapping = filtered_mapping

for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    print(key, ": ", value)

print(len(mapping.items()))

7506424_2_69 :  ['wavs/7506424_2_69.wav', 'cdu csu fraktion frau präsidentin meine sehr geehrten damen und herren liebe kolleginnen und kollegen wo man am meisten drauf erpicht grad das bekommt man meistens nicht liebe kolleginnen und kollegen von den linken und den grünen dieses zitat von wilhelm busch passt perfekt zu ihren anträgen gern erläutere ich warum das so ist sie fordern dass die co kosten allein den vermieterinnen und vermietern aufgebürdet werden sollen und erhoffen sich hierdurch dass die energetische sanierung der gebäude angereizt wird']
7520734_0_99_1 :  ['wavs/7520734_0_99_1.wav', 'hat kann man davon ausgehen dass die regulierung der europäischen union egal ob das jetzt eine verordnung oder eine direktive wird dieselbe richtung gehen wird sie wird detaillierter sein sie wird bestimmte dinge nochmal ausführlicher auch sicher ausbuchstabieren ich halte das deutsche gesetz für eine gute vorbereitung und letzten endes gibt es den deutschen unternehmen da auch ein stück we

In [56]:
import json
from ask_lisa import ask_lisa

def generate_qa(transcript, difficulty):
    instruction = f"""
    Erstelle aus dem folgenden Kontext Frage-Antwort-Paare für drei Komplexitätsstufen.

    - Jede Antwort soll präzise und kurz sein.
    - Stelle die Fragen so, als 
    - Gib das Ergebnis ausschließlich als JSON-Objekt mit den Keys "level_1", "level_2" und "level_3" aus.
    - Unter jedem Key steht wiederum ein JSON-Objekt mit "question" und "answer".
    - Keine anderen Texte oder Erklärungen ausgeben.

    Definition der Komplexitätsgrade:
    1 = sehr einfache, faktische Frage  
    2 = inhaltlich tiefergehende, aber kurze Frage  
    3 = analytische oder interpretierende Frage (z. B. Ursache, Wirkung, Ziel)

    Kontext:
    {transcript}

    Beispielausgabe:
    {{
      "level_1": {{ "question": "Wer wird angesprochen?", "answer": "Linke und Grüne" }},
      "level_2": {{ "question": "Was wird gefordert?", "answer": "CO-Kosten übernehmen" }},
      "level_3": {{ "question": "Warum sollen Vermieter zahlen?", "answer": "Sanierungsanreiz" }}
    }}
    """

    qa_res = ask_lisa(instruction, transcript)

    try:
        qa_all = json.loads(qa_res)
    except json.JSONDecodeError:
        print("⚠️ Antwort war kein gültiges JSON:", qa_res)
        return None, None, difficulty

    key_map = {1: "level_1", 2: "level_2", 3: "level_3"}
    key = key_map.get(difficulty)

    qa = qa_all.get(key, {})
    question = qa.get("question", "")
    answer = qa.get("answer", "")

    return question, answer, difficulty


In [57]:
generate_qa("es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss", 1)

Response status code: 200


('Was wird in dem Kontext gefordert?', 'Mobilitäts- und Verkehrswende', 1)

In [58]:
generate_qa("es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss", 2)

Response status code: 200


('Welche Formen des Protests werden genannt?',
 'Proteste, Initiativen, Self-Made-Projekte, Petitionen und Kundgebungen.',
 2)

In [59]:
generate_qa("es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss", 3)

Response status code: 200


('Warum ist die Vielzahl an Aktivitäten wichtig?',
 'Sie zeigen breite gesellschaftliche Unterstützung und Dringlichkeit für strukturelle Veränderungen',
 3)

# Generating the qa-pairs

In [ ]:
pairs = []
for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    # random number between 1 and 3
    difficulty = random.randint(1, 3)
    print(key)
    qaPair = generate_qa(value[1], difficulty)
    pairs.append({
        'transcript': value[1],
        'qa-pair': qaPair,
        'id': key
    })

# save pairs as json to file
with open("qa_pairs.json", "w", encoding="utf-8") as f:
    json.dump(pairs, f, ensure_ascii=False, indent=4)

In [ ]:
tts = TTS("tts_models/de/thorsten/vits", progress_bar=False)
def to_speech(text):
    tts.tts_to_file(text=text, file_path="output.wav")

In [ ]:
keys = list(mapping.keys())
     
for i in range(1):
    key = random.choice(keys)
    print(mapping[key][1])
    question, answer = generate_qa(mapping[key][1])
    to_speech(question)
    